In [ ]:
'''
gpu fails memory
'''

'\nworks with llmama\nbut the answer are incorrect\n'

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache
import os

b:\Work\Code\p\cag\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model_name = "../model/bitnet-b1.58-2B-4T"
model_name = "B:\\Work\\Code\\f\\4\\testing\\llm\\models\\unsloth-Llama-3.2-1B-Instruct"
input_file_path = "../input/text_1.txt"

In [4]:
# Minimal generate function for token-by-token generation
def generate(model, input_ids: torch.Tensor, past_key_values, max_new_tokens: int = 50) -> torch.Tensor:
    device = model.model.embed_tokens.weight.device
    origin_len = input_ids.shape[-1]
    input_ids = input_ids.to(device)
    output_ids = input_ids.clone()
    next_token = input_ids

    with torch.no_grad():
        for _ in range(max_new_tokens):
            out = model(
                input_ids=next_token,
                past_key_values=past_key_values,
                use_cache=True
            )
            logits = out.logits[:, -1, :]
            token = torch.argmax(logits, dim=-1, keepdim=True)
            output_ids = torch.cat([output_ids, token], dim=-1)
            past_key_values = out.past_key_values
            next_token = token.to(device)

            if model.config.eos_token_id is not None and token.item() == model.config.eos_token_id:
                break

    # Return just the newly generated part
    return output_ids[:, origin_len:]

### DynamicCache Setup
Initializing the DynamicCache mechanism for storing and reusing the model’s key/value states. It also provides a clean_up function to truncate any extra tokens appended by user queries.

In [5]:
torch.serialization.add_safe_globals([DynamicCache])
torch.serialization.add_safe_globals([set])

def get_kv_cache(model, tokenizer, prompt: str) -> DynamicCache:
    # Encode prompt
    device = model.model.embed_tokens.weight.device
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    cache = DynamicCache()

    with torch.no_grad():
        _ = model(
            input_ids=input_ids,
            past_key_values=cache,
            use_cache=True
        )
    return cache

def clean_up(cache: DynamicCache, origin_len: int):
    # Remove any tokens appended to the original knowledge
    for i in range(len(cache.key_cache)):
        cache.key_cache[i] = cache.key_cache[i][:, :, :origin_len, :]
        cache.value_cache[i] = cache.value_cache[i][:, :, :origin_len, :]

### Load model

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
import torch._dynamo
torch._dynamo.config.suppress_errors = True
model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="cuda", # <==== TODO change this
        )

device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to("cpu") # TODO change this to cuda
print(f"Loaded {model_name}.")

Loaded B:\Work\Code\f\4\testing\llm\models\unsloth-Llama-3.2-1B-Instruct.


### Create a Knowledge Prompt 
read the file, build a short system/user prompt, and call `get_kv_cache`.

In [11]:
# with torch.no_grad():
#     torch.cuda.empty_cache()
import torch
torch.cuda.empty_cache()

In [12]:
%%time
with open(input_file_path, "r", encoding="utf-8") as f:
    doc_text = f.read()

system_prompt = f"""
<|system|>
You are an assistant who provides concise factual answers.
<|user|>
Context:
{doc_text}
""".strip()

# Build the cache
doc_cache = get_kv_cache(model, tokenizer, system_prompt)
origin_len = doc_cache.key_cache[0].shape[-2]
print("KV cache built.")

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.07 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 2.77 GiB is allocated by PyTorch, and 112.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
#TODO save the cache and invoke it for inferencing

### Ask Questions Reusing the Cache
We use the same knowledge (no real-time retrieval).

In [ ]:
%%time
# 1st query
question1 = "How did the rain tasted in veridia?"
clean_up(doc_cache, origin_len)
input_ids_q1 = tokenizer(question1 + "\n", return_tensors="pt").input_ids.to(device) # TODO change this to cuda
gen_ids_q1 = generate(model, input_ids_q1, doc_cache)
answer1 = tokenizer.decode(gen_ids_q1[0], skip_special_tokens=True)
print("Q1:", question1)
print("A1:", answer1)

In [ ]:
%%time
# 2nd query
question2 = "What is Aerilon?"
clean_up(doc_cache, origin_len)
input_ids_q2 = tokenizer(question2 + "\n", return_tensors="pt").input_ids.to(device) # TODO change this to cuda
gen_ids_q2 = generate(model, input_ids_q2, doc_cache)
answer2 = tokenizer.decode(gen_ids_q2[0], skip_special_tokens=True)
print("Q2:", question2)
print("A2:", answer2)

In [ ]:
%%time
question3 = "What is Chronarium?"
clean_up(doc_cache, origin_len)
input_ids_q3 = tokenizer(question3 + "\n", return_tensors="pt").input_ids.to(device) # TODO change this to cuda
gen_ids_q3 = generate(model, input_ids_q3, doc_cache)
answer3 = tokenizer.decode(gen_ids_q3[0], skip_special_tokens=True)
print("Q3:", question3)
print("A3:", answer3)